In [100]:
import requests
import pandas as pd
import numpy as np
country = "italy"

In [12]:
import requests
url = 'https://en.wikipedia.org/wiki/Template:COVID-19_pandemic_data/Italy_medical_cases'

In [112]:
provinces = ['VDA', 'LIG', 'PIE', 'LOM', 'VEN', 'TN', 'BZ', 'FVG', 'EMR',
       'MAR', 'TOS', 'UMB', 'LAZ', 'ABR', 'MOL', 'CAM', 'BAS', 'PUG', 'CAL',
       'SIC', 'SAR']

In [159]:
prov_pops = {
    "VEN": 4905854,
    "LOM": 10060574,
    "LAZ": 5879082,
}

In [131]:
df_raw = pd.read_html(url, attrs={"class": "wikitable mw-collapsible"})

In [170]:
df = df_raw[0].copy()

df.columns = df.columns.droplevel(0)

df = df[['Date', 'VDA', 'LIG', 'PIE', 'LOM', 'VEN', 'TN', 'BZ', 'FVG', 'EMR',
       'MAR', 'TOS', 'UMB', 'LAZ', 'ABR', 'MOL', 'CAM', 'BAS', 'PUG', 'CAL',
       'SIC', 'SAR']]

df = df[:-7]

df = df.replace(r"\([^][]*\)", "", regex=True)
df = df.replace(r",", "", regex=True)

df['Date'] = df['Date'].str.replace(".", "-")
df['Date'] = df['Date'].astype('datetime64[ns]')

df = df.fillna(0)

In [171]:
provinces = list(prov_pops.keys())
df = df[provinces]

In [172]:
for province in provinces:
    df[province] = pd.to_numeric(df[province], errors='coerce', downcast='integer')
    
    df[f'{province} DCRA'] = df[province].rolling(7).mean()  # Daily Cases Rolling Average
    
    df[f'{province} DCRA Per Capita'] = df[f'{province} DCRA']/prov_pops[province] * 100000

In [173]:
df

,VEN,LOM,LAZ,VEN DCRA,VEN DCRA Per Capita,LOM DCRA,LOM DCRA Per Capita,LAZ DCRA,LAZ DCRA Per Capita
0,0.0,0.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,15.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,16.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,7.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
242,800.0,2975.0,1198.0,621.142857,12.661259,1963.571429,19.517489,728.285714,12.387745
243,502.0,1687.0,939.0,646.000000,13.167942,2105.142857,20.924679,806.000000,13.709623
244,490.0,2023.0,1224.0,646.714286,13.182502,2239.857143,22.263711,898.142857,15.276923
245,1422.0,4125.0,1219.0,756.000000,15.410161,2565.714286,25.502663,994.714286,16.919551


In [60]:
df.columns

Index(['Date', 'VDA', 'LIG', 'PIE', 'LOM', 'VEN', 'TN', 'BZ', 'FVG', 'EMR',
       'MAR', 'TOS', 'UMB', 'LAZ', 'ABR', 'MOL', 'CAM', 'BAS', 'PUG', 'CAL',
       'SIC', 'SAR', 'New', 'Total', 'New', 'Total', 'ICU', 'Total', 'New',
       'Total', 'Total', 'Refs', 'Ns'],
      dtype='object')

In [61]:
df.head(10)

,Date,VDA,LIG,PIE,LOM,VEN,TN,BZ,FVG,EMR,...,Total,New,Total,ICU,Total,New,Total,Total,Refs,Ns
0,2020-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2,–,–,–,2,–,–,NaN,[2],NaN
1,2020-02-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3,–,–,–,3,–,–,NaN,[3],[a]
2,2020-02-21,NaN,NaN,NaN,15,(1) 2,NaN,NaN,NaN,NaN,...,20,1,1,–,19,–,–,NaN,[4],NaN
3,2020-02-22,NaN,NaN,1,(1) 40,16,NaN,NaN,NaN,2,...,79,1,2,18,75,–,–,NaN,[5][6],NaN
4,2020-02-23,NaN,NaN,2,(1) 57,7,NaN,NaN,NaN,7,...,150,1,3,25,147,–,–,NaN,[7][8],[b]
5,2020-02-24,NaN,NaN,NaN,(4) 61,7,NaN,NaN,NaN,9,...,227,4,7,27,199,1,1,NaN,[9][10],[b]
6,2020-02-25,NaN,1,NaN,(2) 67,(1) 11,NaN,1,NaN,8,...,320,3,10,35,309,–,1,">8,600",[11][12],[b]
7,2020-02-26,NaN,10,NaN,(1) 65,28,NaN,NaN,NaN,(1) 21,...,445,2,12,36,430,2,3,9587,[13][14],[b][c]
8,2020-02-27,NaN,8,1,(5) 98,40,NaN,NaN,NaN,50,...,650,5,17,56,588,42,45,12014,[16][17],[d]
9,2020-02-28,NaN,NaN,9,(3) 128,40,NaN,NaN,NaN,(1) 48,...,888,4,21,64,821,1,46,15695,[19][20],NaN


In [26]:
# aggressive data cleaning


AttributeError: 'DataFrame' object has no attribute 'str'